## Google Geocoding API Pull

[Google's Geocoding API]([https://developers.google.com/maps/documentation/geocoding/overview]) is a service that accepts a place as an address, latitude and longitude coordinates, or Place ID. It converts the address into latitude and longitude coordinates and a Place ID, or converts latitude and longitude coordinates or a Place ID into an address.

In [29]:
import pandas as pd
import googlemaps
import numpy as np

from math import cos, asin, sqrt

### Importing address file

In [3]:
#importing address list from modeling notebook
address_df = pd.read_csv("cleaned_data_exports/address_for_googlemaps.csv")

In [7]:
address_df.drop(columns="Unnamed: 0", inplace=True)

In [9]:
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   full_address  325 non-null    object
 1   ski_resort    325 non-null    object
dtypes: object(2)
memory usage: 5.2+ KB


In [11]:
address_df.head()

,full_address,ski_resort
0,"P.O. Box 166, Chewelah, Washington",49 Degrees North
1,"6600 Peller Avenue South, Hastings, Minnesota",Afton Alps
2,"POB 1068, Snoquale Pass, Washington",Alpental
3,"10620 Mayfield, Chesterland, Ohio",Alpine Valley Ohio
4,"P.O. Box 615, East Troy, Wisconsin",Alpine Valley Wisconsin


### Pulling from API

In [4]:
# initialize the client with your API key

client_key = ""

gmaps = googlemaps.Client(key=client_key)

In [5]:
lat_list = []
long_list = []

for address in address_df['full_address']:
    # geocoding addresses
    geocode_result = gmaps.geocode(address)
    
    # finding lat and lon
    if geocode_result:
        lat_list.append(geocode_result[0]['geometry']['location']['lat'])
        long_list.append(geocode_result[0]['geometry']['location']['lng'])
        
        #sanity check of values
        print(f"Address: {address} - Latitude: {lat}, Longitude: {lng}")
    else:
        print(f"No geocode result found for address: {address}")

# saving back to original dataframe
address_df['latitude'] = lat_list
address_df['longitude'] = long_list

In [12]:
#exporting results as .csv
#file_path = "data/mountain_lat_long.csv" #add file path here
#address_df.to_csv(file_path)

### Airport Data

I will also be including airport information that will be added to the final web application to find the closest airport to each ski resort. 

This information was found on [kaggle](https://www.kaggle.com/datasets/nancyalaswad90/us-airports). I adapted code from [stack overflow](https://stackoverflow.com/questions/69174417/pandas-find-nearest-lat-long-pair).

The haversince formula was used to compute the distance between airport and resort locations. This formula computes the distance between two points on a sphere, and is used in navigation.

In [5]:
resort_location_df = pd.read_csv("data/cleaned_data_exports/mountain_lat_long.csv")
airport_df = pd.read_csv("data/airports.csv")

In [13]:
#dropping column
resort_location_df.drop(columns="Unnamed: 0", inplace=True)

#renaming columns
resort_location_df.rename(columns={"latitude":"lat", "longitude": "long"}, inplace=True)

In [163]:
airport_df.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


### Defining distance function

In [11]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

### Finding Closest Airport

In [67]:
new_df = pd.DataFrame(columns = ["Airport","Location","Distance"])

for _, airport_row in closest_airport_df.iterrows():
    id1 = airport_row['airport']
    lat1 = airport_row['lat']
    lon1 = airport_row['long']
    
    for _, resort_row in resort_location_df.iterrows():
        id2 = resort_row['ski_resort']
        lat2 = resort_row['lat']
        lon2 = resort_row['long']
        
        dist = distance(lat1, lon1, lat2, lon2)
        
        new_df = new_df.append({"Airport": id1, "Location": id2, "Distance": dist}, ignore_index=True)

# Find the closest airport for each resort
closest_airports = new_df.groupby('Location')['Distance'].idxmin()
closest_airports_df = new_df.loc[closest_airports]

#saving as csv
#closest_airports_df.to_csv("data/cleaned_data_exports/closest_airports.csv")

In [136]:
closest_airports_df.rename(columns={"Airport": "airport_1", "Location": "ski_resort", "Distance":"distance_"}, inplace=True)

In [164]:
#renaming columns
closest_airports_df.rename(columns={"airport_1":"airport"}, inplace=True)

In [179]:
#closest_airports_df.to_csv("data/cleaned_data_exports/airport_dist_1.csv")

In [182]:
airport_1_df = pd.merge(closest_airports_df, airport_df[['airport','lat', 'long']],on='airport', how='left')

In [220]:
airport_1_df.rename(columns={"lat":"latitude", "long":"longitude"}, inplace=True)

In [222]:
airport_1_df.to_csv("data/cleaned_data_exports/airport_1_cleaned.csv")

### Finding 2nd Closest Airport

In [100]:
#making a new list for the third airport pull that removes the last airports
airport_list = closest_airports_df['Airport'].to_list()

#dropping values in list
airport_df_2 = airport_df[~airport_df['airport'].isin(airport_list)]

In [122]:
new_df_two = pd.DataFrame(columns = ["airport","ski_resort","distance"])

for _, airport_row in airport_df_2.iterrows():
    id1 = airport_row['airport']
    lat1 = airport_row['lat']
    lon1 = airport_row['long']
    
    for _, resort_row in resort_location_df.iterrows():
        id2 = resort_row['ski_resort']
        lat2 = resort_row['lat']
        lon2 = resort_row['long']
        
        dist = distance(lat1, lon1, lat2, lon2)
        
        new_df_two = new_df_two.append({"airport": id1, "ski_resort": id2, "distance": dist}, ignore_index=True)

# Find the closest airport for each resort
closest_airports_two = new_df_two.groupby('ski_resort')['distance'].idxmin()
closest_airports_df_two = new_df_two.loc[closest_airports_two]

In [183]:
#renaming columns
closest_airports_df_two.rename(columns={"airport_2": "airport"}, inplace=True)

In [185]:
airport_2_df = pd.merge(closest_airports_df_two, airport_df[['airport','lat', 'long']],on='airport', how='left')

In [213]:
airport_2_df.rename(columns={"distance_2": "distance"}, inplace=True)

In [209]:
airport_2_df.rename(columns={"lat_2":"latitude", "long_2":"longitude"}, inplace=True)

In [215]:
airport_2_df.to_csv("data/cleaned_data_exports/airport_2_cleaned.csv")

In [180]:
closest_airports_df_two.to_csv("data/cleaned_data_exports/airport_dist_2.csv")

### Finding 3rd Closest Airport

In [125]:
#making a new list for the third airport pull that removes the last airports
airport_list_3 = closest_airports_df_two['airport'].to_list()

#dropping values in list for new df
airport_df_3 = airport_df_2[~airport_df_2['airport'].isin(airport_list_3)]

In [208]:
airport_df_3

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777
3375,ZZV,Zanesville Municipal,Zanesville,OH,USA,39.944458,-81.892105
3376,Z55,Lake Louise,Lake Louise,AK,USA,62.293689,-146.579422
3377,Z73,Nelson Lagoon,Nelson Lagoon,AK,USA,56.007536,-161.160367


In [128]:
new_df_three = pd.DataFrame(columns = ["airport_3","ski_resort","distance"])

for _, airport_row in airport_df_3.iterrows():
    id1 = airport_row['airport']
    lat1 = airport_row['lat']
    lon1 = airport_row['long']
    
    for _, resort_row in resort_location_df.iterrows():
        id2 = resort_row['ski_resort']
        lat2 = resort_row['lat']
        lon2 = resort_row['long']
        
        dist = distance(lat1, lon1, lat2, lon2)
        
        new_df_three = new_df_three.append({"airport_3": id1, "ski_resort": id2, "distance": dist}, ignore_index=True)

# Find the closest airport for each resort
closest_airports_three = new_df_three.groupby('ski_resort')['distance'].idxmin()
closest_airports_df_three = new_df_two.loc[closest_airports_three]

In [189]:
#renaming columns for merging
closest_airports_df_three.rename(columns={"airport_3":"airport"}, inplace=True)

In [190]:
airport_3_df = pd.merge(closest_airports_df_three, airport_df[['airport','lat', 'long']],on='airport', how='left')

In [191]:
airport_3_df.rename(columns={"airport":"airport_3"}, inplace=True)

In [195]:
airport_3_df.rename(columns={"lat":"lat_3", "long":"long_3"}, inplace=True)

In [181]:
#closest_airports_df_three.to_csv("data/cleaned_data_exports/airport_dist_3.csv")

### Merging dataframes and exporting csv

In [197]:
#merging dataframes on ski resort
final_airport_df = pd.merge(airport_1_df, airport_2_df, on='ski_resort')

final_airport_df = pd.merge(final_airport_df, airport_3_df, on='ski_resort')

In [199]:
# shift column 'C' to first position
first_column = final_airport_df.pop('ski_resort')
  
# insert column using insert(position,column_name,first_column) function
final_airport_df.insert(0, 'ski_resort', first_column)

#saving to csv
final_airport_df.to_csv("data/cleaned_data_exports/closest_airports.csv")

In [201]:
final_airport_df.to_csv("data/cleaned_data_exports/closest_airports.csv")

In [207]:
final_airport_df.loc[final_airport_df['ski_resort'] == "Heavenly Mountain"]

,ski_resort,airport_1,distance_1,lat,long,airport_2,distance_2,lat_2,long_2,airport_3,distance_3,lat_3,long_3
135,Heavenly Mountain,Lake Tahoe,9.002929,38.893882,-119.995335,Minden-Tahoe,16.879594,39.000309,-119.750806,Mojave,462.096824,35.059365,-118.151856


In [161]:
pd.merge(final_airport_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332 entries, 0 to 331
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ski_resort  332 non-null    object 
 1   airport_1   332 non-null    object 
 2   distance_1  332 non-null    float64
 3   airport_2   332 non-null    object 
 4   distance_2  332 non-null    float64
 5   airport_3   332 non-null    object 
 6   distance_3  332 non-null    float64
dtypes: float64(3), object(4)
memory usage: 20.8+ KB
